30 min

<center><h1>PyCon 2017</h1></center>
<center><h2>Portland, Oregon</h2></center>




- we are going to talk about fuzzy search.
- It's Approximate string matching
- Why? Sometime we don't really know what we are looking for

** Fuzzy Search **    
Approximate string matching

- Why
    - Abbreviation: what does "LGTM" mean?
    - Misspellings: "I sware" or "I swear"?
    - Lack of whitespaces: BEDBATHANDBEYOND
    - Cut-off words
    - etc.


- Both US and USA point to the same entity: United States
- Fuzzy search usually based on some sort of string distance

- Applications
    - Entity resolution
        - "US" and "USA"
    - Search Engines
    - Auto correct
- Search based on String distance
    - Quantify qualitative data for analytical purpose

**Soundex**

- phonetic algorithm
- index by sound as pronounced in English
- assigns a soundex coding 
- ideal for spelling inconsistencies

**American Soundex Coding**

[http://www.archives.gov/research/census/soundex.html](http://www.archives.gov/research/census/soundex.html)

every soundex code is a letter and three numbers

| Number | Letter |
|---------|----------|
|1|B,F,P,V|
|2|C,G,J,K,Q,S,X,Z|
|3|D,T|
|4|L|
|5|M,N|
|6|R|

Ignore A,E,I,O,U,H,W,Y

In [52]:
import jellyfish as j

a=j.soundex('PORTLAND')
print(a)
b=j.soundex('PRTLAND')
print(b)

P634
P634


In [5]:
# Numbers and Letters
a=j.soundex('Word #123')
print(a)
b=j.soundex('Word')
print(b)

W630
W630


In [7]:
a=j.soundex('accept')
print(a)
a2=j.soundex('except')
print(a2)

A213
E213


In [8]:
#homophones
a=j.soundex('ado')
print(a)
a2=j.soundex('adieu')
print(a2)

A300
A300


In [9]:
a=j.soundex('forth')
print(a)
a2=j.soundex('fourth')
print(a2)

F630
F630



Soundex with PostgreSQL

https://www.postgresql.org/docs/9.1/static/fuzzystrmatch.html

`CREATE EXTENSION fuzzystrmatch;`

In [10]:
from sqlalchemy import create_engine

engine = create_engine('postgresql://jiaqi@localhost/pycon')
connection = engine.connect()

In [13]:
from sqlalchemy.sql import text

query = "select soundex('Anne'), soundex('Ann'), difference('Anne','Ann')"
res = engine.execute(text(query))
res.fetchall()

[('A500', 'A500', 4)]

Create Table & Load Data

`Create table pypi 
(packages varchar, description varchar);`

`COPY pypi from 'pypi.csv' delimiter ',' csv;`

In [17]:
query = """
        SELECT    packages 
        FROM      pypi 
        WHERE     soundex(packages) = soundex('fuzzysearch')"""
res = engine.execute(text(query))
res.fetchall()

[('facehugger\xa00.1.6',),
 ('face_scrambler\xa00.0.2',),
 ('fakechroot\xa00.2.1',),
 ('fake-useragent\xa00.1.7',),
 ('fakeUserData\xa01',),
 ('Focusr\xa00.3',),
 ('fuzzysearch\xa00.3.0',),
 ('FuzzySortedDict\xa01.0.1',),
 ('fzysearch\xa00.3',)]

In [63]:
query = """
        SELECT    description
        FROM      pypi 
        WHERE     soundex(description) = soundex('fuzzysearch')"""
res = engine.execute(text(query))
res.fetchall()

[('Fake User-Agents of many real devices',),
 ('Fuchsia reduces differential equations for Feynman master integrals to canonical form',),
 ('fuzzysearch is useful for finding approximate subsequence matches',),
 ('fuzzy searches a string input to a list and returns results from the list in order of relevance',),
 ('FPGA core/image generator and FPGA communication',),
 ('Fuzzy search with the PostgreSQL trigram extension',),
 ('Fuzzy search with the PostgreSQL trigram extension',),
 ('Fuzzy search with the PostgreSQL trigram extension',),
 ('Fake servers for testing',),
 ('Fake servers for testing',),
 ('Fake servers for testing',),
 ('Fake servers for testing',),
 ('Fake servers for testing',),
 ('Fake servers for testing',),
 ('Fake servers for testing',)]

In [23]:
query = """
        SELECT    description
        FROM      pypi 
        WHERE     difference(description, 'fuzzysearch') > 2 limit 10"""
res = engine.execute(text(query))
res.fetchall()

[('199fix exception logger for Django',),
 ('quick sort',),
 ('Fisica para Jogoso FGA',),
 ('Asychronous downloads made easy',),
 ('HashiCorp Vault API client',),
 ('Fix skew in images',),
 ('Caches your Django ORM queries and automatically invalidates them.',),
 ('Flash remoting framework. Includes support for NetConnection, RemoteObject, IExternizeable, Flex Producer/Consumer messaging, custom type serialization, and a C-based AMF encoder/decoder.',),
 ('This is wrapper for Apple Push Notification Service.',),
 ('Check script scheduler',)]

**Soundex**
- Soundex is pretty easy to implement
- Computationally fast
- only works on ASCII characters (no foreign languages)
- How do you calculate distance

**Levenshtein distance**
- also call edit distance
- accounts for how many characters you have to change to have the same string
- computationally fast (can handle real time processing)
- pairwise comparison

In [24]:
import Levenshtein as l

l.distance('SMYTHE', 'SMITH')

2

In [55]:
l.distance('pypi', 'pypy')

1

Pitfall: Comparing Addresses

In [63]:
str99 = '99 Broadway'
str100 = '100 Broadway'
str999 = '999 Broadway'

l.distance(str99, str100)

3

In [64]:
l.distance(str99, str999)

1

In [65]:
l.distance(str999, str100)

3

Longer Strings

In [66]:
str1='Mike\'s New York Deli and co'
str2='Sam\'s New York Deli and co'
l.distance(str1,str2)

4

In [67]:
import Levenshtein as l
str1='Mike\'s Deli'
str2='Sam\'s Deli'
l.distance(str1,str2)

4

**Levenshtein**
- counting raw edits penalizes long strings: use a ratio of edits to length
- weighing numbers differently from letters

N-grams

In [26]:
def ngram(tokens, n):
    grams =[tokens[i:i+n] for i in range(len(tokens)-(n-1))]
    return grams

In [27]:
sentence_gram = "The quick brown fox jumped over a lazy dog".split()
grams = ngram(sentence_gram, 3)

for gram in grams:
    print(gram)

['The', 'quick', 'brown']
['quick', 'brown', 'fox']
['brown', 'fox', 'jumped']
['fox', 'jumped', 'over']
['jumped', 'over', 'a']
['over', 'a', 'lazy']
['a', 'lazy', 'dog']


In [30]:
word_gram = "pycon2017"
grams = ngram(word_gram, 3)

for gram in grams:
    print(gram)

pyc
yco
con
on2
n20
201
017


**Scoring Similarity: Jaccard similarity**

intersection over the union 

In [31]:
def get_sim(a_tri,b_tri):
    intersect = len(set(a_tri) & set(b_tri))
    union = len(set(a_tri) | set(b_tri))
    return float(intersect)/(union)

In [32]:
print(grams)
get_sim(grams, grams)

['pyc', 'yco', 'con', 'on2', 'n20', '201', '017']


1.0

In [35]:
a_gram = ngram('aabcccdeeeffgghhh', 3)
b_gram = ngram('abcccdeeffgghhh', 3)
get_sim(a_gram, b_gram)

0.8666666666666667


Trigam Search with Postgres

https://www.postgresql.org/docs/9.1/static/pgtrgm.html

`create extension pg_trgm;`

In [36]:
from sqlalchemy import create_engine

engine = create_engine('postgresql://jiaqi@localhost/pycon')
connection = engine.connect()

In [37]:
query_des="""
    SELECT 
                a.description, 
                similarity(lower(a.description), :descript) as similarity 
    FROM        pypi as a 
    WHERE       lower(a.description) % :descript
    ORDER BY    similarity DESC"""

In [72]:
from sqlalchemy.sql import text

description = 'fuzzy search'
res = engine.execute(text(query_des), descript=description)

In [73]:
descriptions = res.fetchall()
descriptions

[('All Search', 0.411765),
 ('All Search', 0.411765),
 ('Fuzzy path searching for shells', 0.387097),
 ('Fuzzy path searching for shells', 0.387097),
 ('Fuzzy path searching for shells', 0.387097),
 ('Fuzzy path searching for shells', 0.387097),
 ('Media search', 0.368421),
 ('Kagiso Search', 0.35),
 ('fuzzy.ai API', 0.315789),
 ('Search forms for django', 0.307692),
 ('Search in google', 0.304348),
 ('Stabilizer Search', 0.304348),
 ('fuzzy.io API', 0.3)]

In [88]:
query_package="""
    SELECT 
                a.packages, 
                similarity(lower(a.packages), :p) as similarity 
    FROM        pypi as a 
    WHERE       lower(a.packages) % :p
    ORDER BY    similarity DESC"""

In [89]:
pk = "fuzzysearch"
res = engine.execute(text(query_package), p=pk)
packages = res.fetchall()
packages

[('fuzzysearch\xa00.3.0', 0.75),
 ('simple-fuzzysearch\xa00.0.5', 0.521739),
 ('fzysearch\xa00.3', 0.444444),
 ('c2.search.fuzzy\xa01.0a5', 0.416667),
 ('fuzzyset\xa00.0.9', 0.388889),
 ('mysearch\xa01.1', 0.352941),
 ('Fuzzy\xa01.1', 0.333333),
 ('mysearch\xa01.5', 0.315789),
 ('pYsearch\xa03.1', 0.315789)]

In [ ]:
for elem in packages:
    if elem >=

heuristics = for both fields over 
heuristics - well different situations have differnet codes - statistical model leveraging the trigram score as a feature
(back to slides here)

**Other Similarity Metrics**
- NLTK: wordnet
- Word2Vec: uses cosine distance
    - cosine distance between two vectors


In [ ]:
from nltk.corpus import wordnet


In [84]:
word1 = wordnet.synsets("blue")
word2 = wordnet.synsets("green")

In [85]:
word1[0].wup_similarity(word2[0])

0.875

In [48]:
#sample data set from: http://mattmahoney.net/dc/text8.zip
import word2vec
word2vec.word2phrase('text/text8', 'text/text8-phrases', verbose=True)
word2vec.word2vec('text/text8-phrases', 'text/text8.bin', size=100, verbose=True)

b'Starting training using file text/text8\n'b'Words processed: 100K     Vocab size: 72K  \rWords processed: 200K     Vocab size: 130K  \rWords processed: 300K     Vocab size: 181K  \rWords processed: 400K     Vocab size: 231K  \rWords processed: 500K     Vocab size: 278K  \rWords processed: 600K     Vocab size: 323K  \rWords processed: 700K     Vocab size: 366K  \rWords processed: 800K     Vocab size: 407K  \rWords processed: 900K     Vocab size: 450K  \rWords processed: 1000K     Vocab size: 489K  \rWords processed: 1100K     Vocab size: 525K  \rWords processed: 1200K     Vocab size: 562K  \rWords processed: 1300K     Vocab size: 599K  \rWords processed: 1400K     Vocab size: 634K  \rWords processed: 1500K     Vocab size: 669K  \rWords processed: 1600K     Vocab size: 706K  \rWords processed: 1700K     Vocab size: 741K  \rWords processed: 1800K     Vocab size: 778K  \rWords processed: 1900K     Vocab size: 812K  \rWords processed: 2000K     Vocab size: 848K  \rWords processed: 2100K  

b'Vocab size (unigrams + bigrams): 2419827\n'b'Words in train file: 17005206\n'b'Words written: 100K\rWords written: 200K\rWords written: 300K\rWords written: 400K\rWords written: 500K\rWords written: 600K\rWords written: 700K\rWords written: 800K\rWords written: 900K\rWords written: 1000K\rWords written: 1100K\rWords written: 1200K\rWords written: 1300K\rWords written: 1400K\rWords written: 1500K\rWords written: 1600K\rWords written: 1700K\rWords written: 1800K\rWords written: 1900K\rWords written: 2000K\rWords written: 2100K\rWords written: 2200K\rWords written: 2300K\rWords written: 2400K\rWords written: 2500K\rWords written: 2600K\rWords written: 2700K\rWords written: 2800K\rWords written: 2900K\rWords written: 3000K\rWords written: 3100K\rWords written: 3200K\rWords written: 3300K\rWords written: 3400K\rWords written: 3500K\rWords written: 3600K\rWords written: 3700K\rWords written: 3800K\rWords written: 3900K\rWords written: 4000K\rWords written: 4100K\rWords written: 4200K\rWord

In [49]:
import word2vec
model = word2vec.load('text/text8.bin')
model['coffee']

array([ 0.05433614,  0.12524973,  0.07404984, -0.08754835,  0.14115146,
        0.02670596, -0.00371378, -0.0822072 ,  0.13078709,  0.17228104,
        0.04973774,  0.0019975 ,  0.1577177 , -0.05208512, -0.14908025,
       -0.01790344,  0.12454432,  0.02813962,  0.07449716,  0.10513708,
       -0.21924408,  0.01712829, -0.06005951, -0.03893688,  0.17716517,
       -0.0377395 , -0.14821038, -0.01041527,  0.07213477,  0.05798329,
        0.06121416,  0.07777782, -0.05813833, -0.09764528,  0.19129921,
        0.09042751,  0.0471188 ,  0.01570847, -0.05679421,  0.12045967,
       -0.04221033,  0.15452832, -0.13602494, -0.08144774, -0.00420029,
        0.15539469,  0.02450533, -0.05251949,  0.01617594, -0.10412274,
        0.01644405, -0.01289717,  0.15754297, -0.06625356,  0.04333038,
       -0.14395064, -0.03137844,  0.01774955, -0.0368893 , -0.01210278,
       -0.14220341, -0.13698573, -0.03015008,  0.0602311 ,  0.00937867,
        0.02154245, -0.09614298, -0.24017119, -0.07919069,  0.01

In [50]:
def get_similar_words(word):
    indexes, metrics = model.cosine(word)
    return model.generate_response(indexes, metrics).tolist()

In [51]:
get_similar_words('coffee')

[('wheat', 0.8717711986650305),
 ('cotton', 0.8716775132711713),
 ('beef', 0.869811888579094),
 ('tea', 0.8669866678285572),
 ('bananas', 0.8623041870046427),
 ('sugar', 0.856468674871585),
 ('cocoa', 0.8556310923408067),
 ('rice', 0.8497382416920862),
 ('poultry', 0.8476775486170876),
 ('potatoes', 0.8473502626191239)]

In [69]:
indexes, metrics = model.analogy(pos=["coffee", "night"], neg=["day"], n=10)
model.generate_response(indexes, metrics).tolist()

[('tea', 0.3349493584833404),
 ('chicken', 0.3331083672705435),
 ('cigarettes', 0.3314079955476932),
 ('rubber', 0.32838851367457095),
 ('soda', 0.32784664748106473),
 ('tin', 0.3275660350090016),
 ('beans', 0.32454398748329094),
 ('cream', 0.3238316700128127),
 ('cotton', 0.32298672573669773),
 ('rice', 0.3226631851510718)]

In [66]:
#analogy
#sun + cold - warm
indexes, metrics = model.analogy(pos=["sun", "cold"], neg=["warm"], n=10)

In [67]:
model.generate_response(indexes, metrics).tolist()

[('moon', 0.28099868884410184),
 ('earth', 0.2570855551214004),
 ('heavens', 0.25232329108750745),
 ('neptune', 0.2485093399462911),
 ('jupiter', 0.24012433433820843),
 ('orbit_around', 0.2395745361408647),
 ('sky', 0.23716435455645374),
 ('saturn', 0.23677076081542567),
 ('solar_system', 0.23490988018948508),
 ('venus', 0.23345560237762197)]